In [1]:
import pandas as pd
from datetime import datetime
import json
import random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import re
import time

with open('pass.json') as file:
    user = json.load(file)

In [25]:
MAIN_URL = 'https://www.linkedin.com'
CIUDADES = ['bogota', 'medellin', 'cali', 'barranquilla', 'bucaramanga']
#CIUDADES = ['bogota', 'medellin']
CRITERIOS = ['busqueda', 'busco', 'busca', 'buscan', 'buscando', 'buscamos']
#CRITERIOS = ['busqueda', 'busco']
XPATH_URL_BOX = './/div[@class="entity-result__insights t-12"]/a'
XPATH_NAME = './/div[@class="linked-area cursor-pointer"]//a[@class="app-aware-link"]'
XPATH_SUBIDO = './/div[@class="linked-area cursor-pointer"]//p'
XPATH_TEXT = './/div[@class="entity-result__content-inner-container"]//p'
XPATH_BOXES = '//div[@class="entity-result"]'
XPATH_KEY = 'session_key'
XPATH_PASS ='session_password'
XPATH_BUTTON = 'sign-in-form__submit-button'
PAGES = 2


def links_para_buscar():
    links = {}
    for ciudad in CIUDADES:
        links[ciudad] = []
        for criterio in CRITERIOS:
            for page in range(PAGES):
                links[ciudad].append(f'https://www.linkedin.com/search/results/content/?keywords={ciudad}%20{criterio}&origin=SWITCH_SEARCH_VERTICAL&page={page + 1}')
    return links

def open_sesion():
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    options.add_argument('--no-sandbox')

    time.sleep(1)
    driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)
    driver.get(MAIN_URL)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH , '//button[@class="sign-in-form__submit-button"]')))

    user_name = driver.find_element_by_id(XPATH_KEY)
    user_name.send_keys(user['name'])
    password = driver.find_element_by_id(XPATH_PASS)
    password.send_keys(user['pass'])

    login_button = driver.find_element_by_class_name(XPATH_BUTTON)
    login_button.click()
    time.sleep(random.uniform(10, 15))
    return driver

def extraer_data(boxes, data, ciudad):
    for box in boxes:
        try:
            url = box.find_element_by_xpath(XPATH_URL_BOX).get_attribute('href')
            name = box.find_element_by_xpath(XPATH_NAME).text
            subido = box.find_element_by_xpath(XPATH_SUBIDO).text.replace('•','').strip()
            text_box = box.find_element_by_xpath(XPATH_TEXT).text
            data = data.append({'Organizacion': name, 'Texto': text_box, 'Link': url, 'Tiempo de publicado': subido, 
                                 'Ciudad': ciudad}, ignore_index=True)
        except:
            print('Un registro no guardado')
    return data

def open_links(driver, links, data):
    total_links = len(links) * len(links['bogota'])
    avance = 0
    for ciudad in links:
        for url in links[ciudad]:
            avance += 1
            print(f'Links {avance} de {total_links}')
            try:
                driver.get(url)
                time.sleep(random.uniform(2, 5))
                boxes = driver.find_elements_by_xpath(XPATH_BOXES)
                data = extraer_data(boxes, data, ciudad)
            except:
                print(f'URL: {url} no cargo') 
    return data

def save(data):
    date_name = f'{datetime.today().day}-{datetime.today().month}-{datetime.today().year}'
    data.to_excel(f'Linkedin_{date_name}.xlsx', index=False)

def run(data):
    driver = open_sesion()
    links = links_para_buscar()
    data = open_links(driver, links, data)
    save(data)
    return data
    
if __name__ == '__main__':
    data = pd.DataFrame(columns=['Organizacion', 'Texto', 'Link', 'Tiempo de publicado', 'Ciudad'])
    data = run(data)
    print(data)

<ipython-input-25-68e96cf362ee>:33: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('./chromedriver.exe', chrome_options=options)


Links 1 de 60
Links 2 de 60
Links 3 de 60
Un registro no guardado
Un registro no guardado
Links 4 de 60
Links 5 de 60
Links 6 de 60
Un registro no guardado
Links 7 de 60
Un registro no guardado
Un registro no guardado
Links 8 de 60
Un registro no guardado
Links 9 de 60
Links 10 de 60
Links 11 de 60
Links 12 de 60
Links 13 de 60
Links 14 de 60
Links 15 de 60
Un registro no guardado
Un registro no guardado
Links 16 de 60
Un registro no guardado
Links 17 de 60
Un registro no guardado
Links 18 de 60
Un registro no guardado
Un registro no guardado
Un registro no guardado
Un registro no guardado
Links 19 de 60
Links 20 de 60
Un registro no guardado
Links 21 de 60
Un registro no guardado
Links 22 de 60
Un registro no guardado
Un registro no guardado
Un registro no guardado
Links 23 de 60
Links 24 de 60
Un registro no guardado
Un registro no guardado
Links 25 de 60
Un registro no guardado
Un registro no guardado
Links 26 de 60
Links 27 de 60
Un registro no guardado
Links 28 de 60
Un registro n

In [24]:
data

,Organizacion,Texto,Link,Tiempo de publicado,Ciudad
